# **Computer Vision using FastAI**

This notebook will use the FastAI library to complete a computer vision task. 

I cannot reccommend the FastAI course highly enough. It's a great entry point in to Neural Networks and Deep Learning in general. Many of the explanations I provide below are snippets from the course.

You can find more information here:

https://www.fast.ai/

and the course here:

https://course.fast.ai/

I reccommend using Google Collab for the course, rather the a jupyter notebook.

# **Project: Pneumonia Prediction**

The problem: **Can we correctly classify chest X-Rays of patients that have pneumonia & those that do not?**

# Project plan

Here are the steps I plan to follow in this project:

* Import libraries & data

* Data overview - Show a few images, check classes, length of dataset etc.

* Train a model as a baseline

* Assess model misclassifications

* Model tuning:
    - Plot learning rates
    - Unfreeze the ResNet model
    - Use discriminative learning rates
    
It's also worth noting that I am following the above steps in order to gain more experience with FastAI - the particular steps I take may or may not be neccesary, I just want to utilise the various tools that FastAI offers.


# Importing the libraries

In [ ]:
import random, os
import numpy as np
import torch
from fastai.vision.all import *

# Getting the data

I'll get the path for the dataset. This saves reading in data in the usual way, clogging up your notebook with thosands of image paths.

In [ ]:
path = Path('/kaggle/input/chest-xray-pneumonia/chest_xray/'); path.ls()

DataLoader is a class that provides batches of a few items at a time - let's do that

In [ ]:
data = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

dls = data.dataloaders(path)

In [ ]:
dls.valid.show_batch(max_n=12, nrows=2)

Observations from these images will be noted below. First, I'll do some more checks to confirm our **categories** are just 'Normal' and 'Pneumonia':

In [ ]:
print(dls.vocab)

And our **dataset length**

In [ ]:
len(dls.train_ds), len(dls.valid_ds)

# Observations

To the **untrained** eye it is difficult to say with certainty, looking at an X-ray, what visual distinctions there are between 'Normal' and those with 'Pneumonia'.

Perhaps the larger white mass in the chest is indicative of pneumonia, but this is conjecture on my part.

Also, our validation set is quite a lot smaller than our training set, but this isn't a problem. Actually, we often want our training set to be as large as possible!

# Baseline

Let's train a model.

FastAI has several ResNet models in readily available:

* Resnet18
* Resnet34
* Resnet50
* Resnet101
* Resnet152

They vary in size, with ResNet152 being trained on the most amount of images. 

For the basline, I will use **ResNet18**

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=[error_rate,accuracy])
learn.fine_tune(6)

I'll save this model now

In [ ]:
learn.save('model_1')

**Fantastic results.**

This is often why **training a model quickly is a good place to start**. 

Why? Well, it prompts a number of questions:

* Is this score good enough already? 

* Do we have time to train more models? 

* Is optimization necessary?

These are all questions the may or may not provide direction in your specific domain. 

A fantastic course relevant to my point above is **"Structuting Machine Learning Projects"** by **Andrew Ng** on Coursera. 

It focuses on the practical realities of ML tools & products, optimization, setting a basline, and it is a fantastic resource. 

Here's a link:

https://www.coursera.org/learn/machine-learning-projects



In a medical domain, I'd say it is worth spending time to see if we can optimize further, but in some cases, a model such as the one above would be more than sufficient.

Now, where did we go wrong (and right)?

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

So we can see here that the **model did very well**. 

The main issuse the model has is producing **False Positives**. 20 Patients would be predicted to have pneumonia when in fact they do not. However, the more important metric in this case is **False Negatives**, of which there are 3. Not perfect, but a very good result overall.

We do note that we got some incorrect, we can view these explicitly.

Here, I'll choose the **top 3 losses**

In [ ]:
interp.plot_top_losses(3, nrows=3)

This can be an opportunity to *fix* issues in your dataset. For instance, if an image is incorrectly labelled. 

If you're using scraped data, for instance, this can be a problem. I built a classifier that can classify the Big Cats of Africa, and the amount of domestic cats that snuck in to the dataset was a real headache, so this can be a very important & useful step.

This relies on **domain expertise** though, so for this case I wouldn't attempt it. However, it is curious that the large white mass in the chest area on those incorrectly classified above do look very similar to the test batch we showed earlier.

# Improving the model

Adjusting the learning rate. If our learning rate is too low, it can take an age train our model. In addition, it means that we might overfit, because every time we do a complete pass through the data, we give our model a chance to memorize it. 

Seeing as the model did so well with ResNet18, I will continue to use that.

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=error_rate)
lr_min,lr_steep = learn.lr_find()

In [ ]:
print(f"Minimum/10: {lr_min:.2e}, steepest point: {lr_steep:.2e}")

# Interpreting the Learner

The aim is to pick a learning rate that is not too high, and not too low.

The graph above helps us to ensure we acheive that.

Ideally, we want to pick a point where the Loss is still decreasing, and has not started to increase. Above, that happens at around 10^-1.

The decrease in Loss starts around 10^-4. I will pick between 10^-3 and 10^-2. 

Note that the Learner is on a logarithmic scale, which is why the middle point between 10^-3 and 10^-2 is between 3e-3 and 4e-3. 

Let's **re-train the model** for the same number of epochs but with the **new learning rate.**

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=[error_rate,accuracy])
learn.fine_tune(6, base_lr=5e-3)

Let's save this model, too

In [ ]:
learn.save('model_2')

Fantastic, our model has **improved our accuracy due to changing the learning rate**.

As an aside, the default value that FastAI uses is 1e-3.

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

We can also normalize this plot to view the percentages.

This can be useful to gain perspective. 

For instance, we now see that our **recall is close to 100%** and that **False Negatives only occur 0.02% of the time**

In [ ]:
interp.plot_confusion_matrix(normalize=True, norm_dec=3)

The results are very good. We managed to **reduce the number of False Positives**. 

However the False Negatives are still the same. 

**We did improve the model** though, which was our aim.

These misclassifications are shown below explicitly:

In [ ]:
interp.most_confused(min_val=1)

# Shown is 'ACTUAL, PREDICTED, COUNT'

Again, let's view the **top 3 losses**

In [ ]:
interp.plot_top_losses(3, nrows=3)

# How many epochs to train for?

Your first approach to training should be to simply pick a number of epochs that will **train in the amount of time you're willing to wait**.

Then look at the training and validation loss plots and in particular **your metrics**. If you see that they are still getting better even in your final epochs, then you know that you have not trained for too long.

**Remember, it's not just that we're looking for the validation loss to get worse, but the actual metrics**. This is an important realisation and can change how you approach machine learning & deep learning problems. If your desired metrics are improving, on the validation or test set, then that is the most important factor.

The loss function is just something that we use to allow our optimizer to have something it can differentiate and optimize; it's not actually the thing we care about in practice.

# Unfreezing the model

A pre trained model, such as the ResNet model we are using here, can be used on data other than the data it was trained on - this is **transfer learning**



Because the model was trained on some other dataset, we might be able to improve it by effectivley removing the final linear layer of the model - which is specifically designed to classify the categories in the original pretraining dataset - and replace it with a layer specific to our dataset.


From FastAI:

*We want to train a model in such a way that we allow it to remember all of these generally useful ideas from the pretrained model, use them to solve our particular task, and only adjust them as required for the specifics of our particular task.*

*Our challenge when fine-tuning is to replace the random weights in our added linear layers with weights that correctly achieve our desired task without breaking the carefully pretrained weights and the other layers. There is actually a very simple trick to allow this to happen: tell the optimizer to only update the weights in those randomly added final layers. Don't change the weights in the rest of the neural network at all. This is called freezing those pretrained layers.*

When we call the fine_tune method FastAI does two things:

* Trains the randomly added layers for one epoch, with all other layers frozen
* Unfreezes all of the layers, and trains them all for the number of epochs requested

Let's try that...

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=[error_rate,accuracy])
learn.fit_one_cycle(3, 5e-3)

Next, unfreeze the model...

In [ ]:
learn.unfreeze()

Now we can find a new learning rate

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(5, lr_max=1e-6)

The model didn't perform as well as the models above. 

I will now try another method...

# Discriminative Learning Rates

FastAI lets you pass a Python slice object anywhere that a learning rate is expected. The first value passed will be the learning rate in the earliest layer of the neural network, and the second value will be the learning rate in the final layer. 

I'll select a range here based on the plot above. 

I will then train for a while to see how the model performs. When our scores are already so high, it is hard to improve, so I am not expecting much improvement, if any, but I want to try this additional step

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=[error_rate,accuracy])
learn.fit_one_cycle(3, 5e-3)
learn.unfreeze()
learn.fit_one_cycle(8, lr_max=slice(5e-3,1e-3))

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

We certainly **didn't improve on our earlier results** - although 98% accuracy still can't be considered a poor result!

# Results

We were able to acheive an **accuracy score of over 99%**

This was accompanied by high precision & recall, with close to 100% of those with Pneumonia being detected successfully.

First, I set a baseline - which scored incredibly highly at the outset. From there, I knew it would be a challenge to improve the score.

Next, I adjuted the learning rate of the model and managed to improve the accuracy by nearly a full percentage point. This happens to be our final model.

I also tried unfreezing the base model, and also implementing disriminative learning rates. Though both of these still scored highly, neither matched up to earlier results. 

Overall, I am happy the the model is a success.

# **Thank you for reading**

Please consider upvoting if you found this notebook helpful.

# Some of my other work

**EDA & Prediction using SMOTE [HR Dataset]**

https://www.kaggle.com/joshuaswords/awesome-hr-data-visualization-prediction


**Stroke Prediction & Model Interpretation with SHAP, LIME, and ELI5**

https://www.kaggle.com/joshuaswords/predicting-a-stroke-shap-lime-explainer-eli5


**Beautiful EDA & Clustering [World Happiness Index 2021]**

https://www.kaggle.com/joshuaswords/awesome-eda-2021-happiness-population


**EDA & Gold Price Prediction using Prophet**

https://www.kaggle.com/joshuaswords/eda-gold-price-prediction-prophet
